<a href="https://colab.research.google.com/github/john-decker/Cultural_Heritage/blob/main/Random_Met.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dominate

In [ ]:
import requests
import json
import random
import time
from PIL import Image
from io import BytesIO
import dominate
from dominate.tags import *
from dominate.util import raw
import os
import webbrowser

def get_random_object(id_number):
	'''This function retreives specific information from the Met API and returns the record.
	inputs: string (id number for a specific work)
	returns json file'''

	import json

	met_object_ids = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects")
	specific_example = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects/" + id_number)
	print("Object", id_number, "search status:", specific_example.status_code)

	try:
		object_details = json.loads(specific_example.text)

	except  json.decoder.JSONDecodeError:
		print("Sorry, an error occurred decoding the file. Please try again")

	return object_details

def get_specific_info(info_file):
	'''This function takes in a text file from a JSON.loads operation from the Met API and reduces it to specific information.
	inputs: text file
	returns: dictionary'''

	working_dict ={}
	target_terms = ['objectID', 'accessionYear', 'primaryImageSmall', 'objectName', 'title', 'culture', 'period', 'artistDisplayName', 'objectDate', 'medium']
	for item in info_file:
		if item in target_terms:
			working_dict[item] = info_file[item]
	return(working_dict)



In [ ]:
#open Met API and get list of object ids
met_object_ids = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects")
status = f"Met API Status Code: {met_object_ids.status_code}"


#transform string result from Met API to list
id_array = met_object_ids.text.split(',')

#generate a random index number to select an id
random_object_id = random.randint(0, len(id_array))
random_object_id_string = str(id_array[random_object_id])

try:
	retrieve_met_object = get_random_object(random_object_id_string)

except  json.decoder.JSONDecodeError:
	json_message = "Sorry, an error occurred while retreiving the file. Please try again."
	print(json_message)

#parse info to get specific items for output
new_met_item = get_specific_info(retrieve_met_object)


information_data = {}

if new_met_item['primaryImageSmall']:
	for key, value in new_met_item.items():
		if value:
			information_data[key] = value
			# print(f'{key.capitalize()}: {str(value).capitalize()}')
			image_url = new_met_item['primaryImageSmall']
	image_url = new_met_item['primaryImageSmall']
	#from https://stackoverflow.com/questions/7391945/how-do-i-read-image-data-from-a-url-in-python
	image_file = requests.get(image_url)
	img = Image.open(BytesIO(image_file.content))
	# img.show()
	width, height = img.size

else:
	for key, value in new_met_item.items():
		if value:
			information_data[key] = value
			# print(f'{key.capitalize()}: {str(value).capitalize()}')

	new_met_item['primaryImageSmall'] = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Metropolitan_Museum_of_Art_%28The_Met%29_-_Central_Park%2C_NYC.jpg/512px-Metropolitan_Museum_of_Art_%28The_Met%29_-_Central_Park%2C_NYC.jpg'
	image_url = new_met_item['primaryImageSmall']
	width, height = '512', '512'

Object 193947 search status: 200


In [ ]:
#create dom  using bootstrap
embed_page = f'<iframe src={image_url} name="iframe_a", width={width}, height={height}></iframe>'

doc = dominate.document(title="Random Met Object")

with doc.head:
    link(rel='stylesheet', href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css", integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC", crossorigin="anonymous")

with doc:
	with div(cls="container-fluid", id="main-container"):
		with div(cls="position-absolute top-50 start-50 translate-middle", style="width: 50vw"):
			with div(cls="position-absolute top-50 start-50 translate-middle"):
				h2('Metropolitan Museum of Art')
				p('Random Object Generator')
				with div (cls="col p-2", id='image-embed'):
					p(raw(embed_page))
					if new_met_item['primaryImageSmall'] == 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Metropolitan_Museum_of_Art_%28The_Met%29_-_Central_Park%2C_NYC.jpg/512px-Metropolitan_Museum_of_Art_%28The_Met%29_-_Central_Park%2C_NYC.jpg':
						figcaption('Sorry, no image URL available for this object.')
				with div (cls="col p-3 bg-secondary text-white", id='image-info').add(ul(style="list-style-type: none")):
					for key, value in information_data.items():
						li(f'{key.capitalize()}: {value}')

print(doc)



with open('index.html', 'w') as outfile: #naming it index will allow the next step to automatically ouput to colab
    outfile.write(doc.render())



<!DOCTYPE html>
<html>
  <head>
    <title>Random Met Object</title>
    <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" rel="stylesheet">
  </head>
  <body>
    <div class="container-fluid" id="main-container">
      <div class="position-absolute top-50 start-50 translate-middle" style="width: 50vw">
        <div class="position-absolute top-50 start-50 translate-middle">
          <h2>Metropolitan Museum of Art</h2>
          <p>Random Object Generator</p>
          <div class="col p-2" id="image-embed">
            <p><iframe src=https://images.metmuseum.org/CRDImages/es/web-large/39023.jpg name="iframe_a", width=443, height=625></iframe></p>
          </div>
          <div class="col p-3 bg-secondary text-white" id="image-info">
            <ul style="list-style-type: none">
              <li>Objectid: 193947</li>
              <li>Accessi

In [ ]:
#display to colab from: https://stackoverflow.com/questions/59741453/is-there-a-general-way-to-run-web-applications-on-google-colab
import IPython
from IPython.core.magic import register_line_magic
import subprocess

@register_line_magic
def run_local_server(line):
    handle = IPython.display.display(
            IPython.display.Pretty("Launching my server..."),
            display_id=True,
    )
    subprocess.Popen(['python', '-m', 'http.server'])
    shell = """
        (async () => {
            const url = new URL(await google.colab.kernel.proxyPort(8000, {'cache': true}));
            const iframe = document.createElement('iframe');
            iframe.src = url;
            iframe.setAttribute('width', '100%');
            iframe.setAttribute('height', '1200');
            iframe.setAttribute('frameborder', 0);
            document.body.appendChild(iframe);
        })();
    """
    script = IPython.display.Javascript(shell)
    handle.update(script)

In [ ]:
#use "line magic notation to run server and output"
%run_local_server

<IPython.core.display.Javascript object>